<a href="https://colab.research.google.com/github/Airdef/BeatData_Test/blob/main/LLamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install lib

In [ ]:
!pip install -q llama-index
!pip install openai
!pip install PyPDF2
!pip install transformers
!pip install llama-cpp-python


Import lib

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex,Settings #Load data and store emebddings
from llama_index.llms.openai import OpenAI #chatgpt as llm
from llama_index.embeddings.openai import OpenAIEmbedding #embedding generation
from llama_index.core.node_parser import SentenceSplitter #break text into sentences

import time #wait between inputs
import os
from IPython.display import clear_output #clear console
from getpass import getpass #secure password input

#from PyPDF2 import PdfReader
#from IPython.display import Markdown, display #Display output

Import Openai key and create directory for pdfs if nit exists

In [ ]:

dir="/content/KW_space"
os.environ['OPENAI_API_KEY']=""

while  not os.environ['OPENAI_API_KEY']:
  os.environ['OPENAI_API_KEY']=getpass("Please write OPENAI API KEY: ")
  clear_output(wait=True)
  time.sleep(1)
print("Key saved")

if not os.path.exists(dir):
    try:
      !mkdir KW_space
      print(f"directory: {dir} created")
    except:
      print(f"Fatal Error")
      exit(0)
else:
    print(f"Directory {dir} exists.")


In [ ]:
print("please refresh files index (second icon on the left)\n")
time.sleep(1)

while not os.listdir("/content/KW_space"):
    print("Directory is empty\n")
    print("please Load PDF into dir called: KW_space\n")
    print("If you can not see the directory please refresh the index\n")
    clear_output(wait=True)
    time.sleep(2)
print("File Loaded\n")

Read PDF from directory

In [ ]:
try:
  doc=SimpleDirectoryReader(dir).load_data()
  print("files imported correctly")
except:
  print("error file not imported, control files")


Setup model and chunks sizes



In [ ]:
Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 3900

In [ ]:
index= VectorStoreIndex.from_documents(doc)
del doc
# index.storage_context.persist()    #in case you want to save embeddings into a folder

Q/A

In [ ]:
query_engine=index.as_query_engine()


SOLUTION 1

In [ ]:
count=0
while True:
  if count >0:
    count=0
  Q=input("what is you question based on the pdf?: ")
  if Q == "exit":
    break

  try:
    print("\n",query_engine.query(Q),"\n\n")

  except:
    count+=1
    print("Problem occurred, please try again")
    if count >=5:
      exit(1)